In [1]:
import ee
import datetime
import pandas as pd
import time

# Authenticate and initialize the Earth Engine API
ee.Authenticate()
ee.Initialize()

def get_ndvi(lat, lon, date):
    date = datetime.datetime.strptime(date, '%Y-%m-%d')
    start_date = date - datetime.timedelta(days=365)
    point = ee.Geometry.Point([lon, lat])

    dataset = ee.ImageCollection('MODIS/006/MOD13A1') \
        .filterDate(ee.Date(start_date), ee.Date(date)) \
        .sort('system:time_start', opt_ascending=False) \
        .first()

    if dataset is None:
        return None

    ndvi = dataset.select('NDVI')
    ndvi_value = ndvi.reduceRegion(ee.Reducer.first(), point, 500).get('NDVI')
    
    return ndvi_value.getInfo()

df = pd.read_csv("mapdata_wildfires.csv")
df_error = pd.read_csv("weather_data_wildfires_error.csv")

coordinates_and_dates = []
coordinates_and_dates_error = []

for _, row in df.iterrows():
    lat = row['incident_latitude']
    lon = row['incident_longitude']
    date_created = row['incident_dateonly_created']
    coordinates_and_dates.append((lat, lon, date_created))

for _, row in df_error.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    date_created = row['Date']
    coordinates_and_dates_error.append((lat, lon, date_created))

ndvi_values = []

for lat, lon, date in coordinates_and_dates_error: # Change for full dataset, remove _error
    ndvi_value = get_ndvi(lat, lon, date)
    ndvi_values.append(ndvi_value)

feature_list = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point(lon, lat), {'Latitude': lat, 'Longitude': lon, 'Date': date, 'NDVI': ndvi})
    for (lat, lon, date), ndvi in zip(coordinates_and_dates_error , ndvi_values) # Change for full dataset, remove _error
])

task_config = {
    'folder': 'EarthEngineExports',
    'fileFormat': 'CSV',
    'driveFileNamePrefix': 'vegetation_indices_data_wildfires',
    'selectors': ['Latitude', 'Longitude', 'Date', 'NDVI']  # Add this line to specify which properties to export
}

task = ee.batch.Export.table.toDrive(feature_list, **task_config)
task.start()

while task.status()['state'] in ['READY', 'RUNNING']:
    print('Exporting NDVI data...')
    time.sleep(30)

print('NDVI data export completed.')


Enter verification code: 4/1AVHEtk7AwostHlo5ms7f6QapIn4ZMsIMxwPd_h9y65JxDczTif-LNR9g1c8

Successfully saved authorization token.
Exporting NDVI data...
Exporting NDVI data...
Exporting NDVI data...
Exporting NDVI data...
Exporting NDVI data...
NDVI data export completed.
